In [1]:
songname = 'daechita'

input_info = songname + '_info.txt'
output_allnode = songname + '_allnode.txt'
output_nodelist = songname + '_nodelist.txt'
output_distmat = songname + '_distmat.txt'


f = open(input_info,'r'); l = len(f.readlines())
f = open(input_info,'r')

s = [];
for i in range(l):
    s.append(f.readline())

p = [];
for j in range(l):
    t = s[j].split('"')
    p.append([
        int(t[1]),
        int(t[3]),
        int(t[5])
    ])

r = [
        [ p[0][0], p[0][2] - p[0][1] ]
    ];
a = p[0][2]
for i in range(1,l):
    b = p[i][1]
    if a != b:
        r.append(
            [ 0, b - a ]
        )
    r.append(
        [ p[i][0], p[i][2] - p[i][1] ]
    )
    a = p[i][2]

q = []
#Convert data format.
notes = ['C3', 'C#3', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3', 'C4', 'C#4', 'D4', 'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4', 'C5', 'C#5', 'D5', 'D#5', 'E5', 'F5', 'F#5', 'G5', 'G#5', 'A5', 'A#5', 'B5', 'C6', 'C#6', 'D6', 'D#6', 'E6', 'F6', 'F#6', 'G6', 'G#6', 'A6', 'A#6', 'B6']
pitch_aria = [83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36]
pitch = [131, 139, 147, 156, 165, 175, 185, 196, 208, 220, 233, 247, 262, 277, 294, 311, 330, 349, 370, 392, 415, 440, 466, 494, 523, 554, 587, 622, 659, 698, 740, 784, 831, 880, 932, 988, 1047, 1109, 1175, 1245, 1319, 1397, 1480, 1568, 1661, 1760, 1865, 1976]

for i in range(len(r)):
    pit = r[i][0]
    duration = r[i][1]
    if pit == 0: # rest
        pit = 0
    else:
        for j in range(len(pitch_aria)):
            if pit == pitch_aria[j]:
                pit = pitch[j]
                note  = notes[j]
                break
        else:
            print ('Something is wrong', pit)
    duration = duration/160 
    q.append([pit,duration])
         

#Convert pitch.duration
o = []
for i in range(len(q)):
    o.append(q[i][0] + q[i][1]/100.)

with open(output_allnode,'w') as f:
    for node in o:
        f.write('%s\n' % node)

n = len(o)
nodelist = [o[0]]
for i in range(n):
    if not(o[i] in nodelist):
        nodelist.append(o[i])

with open(output_nodelist,'w') as f:
    for node in nodelist:
        f.write('%s\n' % node)   
        
###############################################################################
### FIND DIST_MAT
        
import networkx as nx
from sympy import zeros
import numpy as np

L = np.loadtxt(output_allnode)

G = nx.Graph()
G.add_nodes_from(L)

for i in range(len(L)-1):
    if L[i] != L[i+1]:
        e0 = L[i]
        e1 = L[i+1]
        if G.has_edge(e0,e1): 
            G[e0][e1]['weight'] += 1
        else:
            G.add_edge(e0,e1,weight=1)
        
# find distance matrix
nodelist = list(G.nodes)

distmat = zeros(len(G.nodes))
for i in range(len(G.nodes)):
    for j in range(i,len(G.nodes)):
        if j == i:
            distmat[i,j] = 0
        else:
            path = nx.shortest_path(G,nodelist[i],nodelist[j]) # shortest_path
            for k in range(len(path)-1):
                distmat[i,j] += 1/G[path[k]][path[k+1]]['weight']
        distmat[j,i] = distmat[i,j]

# write distance matrix to a file
mat = np.matrix(distmat)
with open(output_distmat,'w') as f:
    for line in mat:
        np.savetxt(f,line,fmt='%.15f')

In [2]:
len(nodelist)

56

In [3]:
len(L)

571

In [ ]:
#0.24 이 한 정간